In [1]:
import requests
from bs4 import BeautifulSoup

path = r'https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
response = requests.get(path)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

💡 파라미터가 너무 많은 URL의 경우 꿀팁! 💡
1. 개발자 도구 진입
    Network > 처음 실행된 클릭 > Payload > 파라미터 가져오기
2. 딕셔너리 형태로 변환
    - 숫자형은 그대로, 나머지는 문자열 형태로 변환
    - 빈 파라미터는 그대로 두기

In [2]:
# 결과
# ? 뒤는 삭제

param = {
    'isDetailSearch': 'N',
    'searchGubun': 'true',
    'viewYn': 'OP',
    'strQuery': '패션 인공지능',
    'order': '/DESC',
    'onHanja': 'false',
    'strSort': 'RANK',
    'iStartCount': 0,
    'sflag': 1,
    'isFDetailSearch': 'N',
    'pageNumber': 1,
    'resultKeyword': '패션 인공지능',
    'icate': 're_a_kor',
    'colName': 're_a_kor',
    'pageScale': 100,
    'isTab': 'Y',
    'query': '패션 인공지능'
}

path = r'https://www.riss.kr/search/Search.do'
response = requests.get(path, params=param)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
articles = soup.select('.srchResultListW > ul > li')

### 상세 페이지 크롤링 하는 법
- for문 안쪽에서 추가로 요청을 보낼 수 있음
- 목록에서 클롤링한 링크로 상세 페이지 요청

In [ ]:
# 헤더 없이 soup 출력했을 때 원하는 정보가 모두 나오지 않음
# -> 서버에서 요청을 거절했기 때문
for article in articles[:1]:
    title = article.select_one('.title > a').text
    link = 'https://www.riss.kr' + article.select_one('.title > a').get('href')

    # 상세페이지 요청
    response = requests.get(link)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    press = soup.select_one('.infoDetailL > ul > li:nth-of-type(2) > div').text
    print(title, link, press)

AttributeError: 'NoneType' object has no attribute 'text'

선택자를 올바르게 입력했음에도 불구하고 NoneType 에러 발생
→ 서버에서 요청을 거절했기 때문

서버의 요청 거절에 대한 case 3가지
1. 상태코드(Status Code) 200 → 응답받은 HTML이 올바르게 작동하지 않음
2. 상태코드(Status Code) 400번대
3. 무한루프

헤더를 추가하면 문제해결 가능
- 헤더: 내가 누군지 알려주는 정보
- 서버쪽에서 **요청을 거절할 때**, 필요한 정보를 같이 날려준다.
- 'User-Agent':'Mozilla/5.0'
- 'Referer':'URL 주소값'


In [3]:
# 헤더 추가
# 서버에서 거절했던 요청도 함께 나옴

header = {
    'User-Agent' : 'Mozilla/5.0 ',
    'Referer' : 'https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
}
for article in articles[:1]:
    title = article.select_one('.title > a').text
    link = 'https://www.riss.kr' + article.select_one('.title > a').get('href')

    # 상세페이지 요청
    response = requests.get(link, headers=header)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    print(soup)



















 
 






	
	
	
		
		
	
	
	
	
	
	



<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="no-referrer-when-downgrade" name="referrer">
<meta content="width=device-width" name="viewport"/>
<meta content="telephone=no" name="format-detection"/><!-- 아이폰 전화번호 스타일 초기화 -->
<meta content="Copyright(c) KERIS. ALL RIGHTS  RESERVED" name="copyright"/>
<meta content="" property="og:title"/>
<meta content="한국교육학술정보원(KERIS)에서 제공하는 학술연구정보서비스(RISS)" property="og:description"/>
<meta content="https://www.riss.kr/link?id=A108142420&amp;ssoSkipYN=Y" property="og:url">
<meta content="https://www.riss.kr/commons/images/thumbnailLogo.png" property="og:image"/>
<meta content="200" property="og:image:width"/>
<meta content="300" property="og:image:height"/>
<meta content="website" property="og:type"/>
<title>RISS 검색 - 국내학술지논문 상세보기</title>
<link href="/commons/images/favicon.ico?" rel="icon" type="image/x-icon"

In [4]:
header = {
    'User-Agent' : 'Mozilla/5.0 ',
    'Referer' : 'https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
}
for article in articles[:70]:
    title = article.select_one('.title > a').text
    link = 'https://www.riss.kr' + article.select_one('.title > a').get('href')

    # 상세페이지 요청
    response = requests.get(link, headers=header)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    press = soup.select_one('.infoDetailL > ul > li:nth-of-type(2) > div').text
    print(title, link, press, sep='\n')

패션 제조 기업의 디지털 트랜스포메이션을 위한 인공지능 솔루션 개발 및 활용 현황
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=dcd1a8e873c0ca787ecd42904f0c5d65&keyword=패션 인공지능
한국패션비즈니스학회
패션산업에 반영된 인공지능 사례연구 -패션 소비자 중심으로-
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=9c3313ce986b119147de9c1710b0298d&keyword=패션 인공지능
한국디자인문화학회
텍스트마이닝을 활용한 인공지능과 패션의 융합에 대한 인식변화 분석
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=657a0ea4ce31f352b36097776a77e665&keyword=패션 인공지능
한국전시산업융합연구원
인공지능(AI) 패션 스타일링 서비스 ‘스티치 픽스’의 융합 콘텐츠 연구
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=404996b30e5d827bb7998d826d417196&keyword=패션 인공지능
한국전시산업융합연구원
생성형 인공지능이 적용된 패션 이미지의 활용방안 제안-학습용 인공지능 데이터와 인공지능 기술적용 중심으로-
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b22cdd16f60538efd18150b21a227875&keyword=패션 인공지능
한국디자인문화학회
패션과 인공지능을 융합한 2D 이미지 기반 가상 착의의 복합적 특성 개선 연구
htt

### SSL 에러 해결방법
- SSL 인증 비활성화
- requests.get() 요청보낼 때 옵션 하나 추가
    - verify=False
    - `requests.get(link, headers=header, verify=False)`

SSL 인증 비활성화 시 WARNING 메시지가 뜰 수 있다.
아래의 방법으로 경고메시지 OFF 가능
```python
import urllib3
urllib3.disable_warnings()
```

In [ ]:
## 발행연도, 주제어 크롤링

header = {
    'User-Agent' : 'Mozilla/5.0 ',
    'Referer' : 'https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
}
for article in articles[:15]:
    title = article.select_one('.title > a').text
    link = 'https://www.riss.kr' + article.select_one('.title > a').get('href')

    # 상세페이지 요청
    response = requests.get(link, headers=header)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # 1. 순서를 기준으로 찾기
    press = soup.select_one('.infoDetailL > ul > li:nth-of-type(2) > div').text
    pyear = soup.select_one('.infoDetailL >ul > li:nth-of-type(5) > div').text
    subject = soup.select_one('.infoDetailL >ul >li:nth-of-type(7) > div').text
    
    print(pyear)
    sub_data = []
    for sub in subject.split(';'):
        sub_data.append(sub.strip())
    print(sub_data)

2022
['디지털 트랜스포메이션', '패션산업', '인공지능', 'AI 진화 단계', 'digital transformation', 'fashion business', 'artificial intelligence', 'AI evolutionary stage']
2020
['Artificial Intelligence(인공지능)', 'Fashion Industry(패션산업)', 'Fashion Design (패션 디자인)']
2025
['인공지능', '패션', '인공지능 패션', '텍스트 마이닝', '융합', 'AI', 'Fashion', 'AI Fashion', 'Text Mining', 'Convergence']
2020
['Artificial Intelligence(인공지능)', 'Fashion Styling Service(패션 스타일링 서비스)', 'Stitch Fix(스티치 픽스)', 'Convergence Content(융합 콘텐츠)']
2023
['AI Learning(인공지능 학습)', 'Fashion Generative AI (패션생성형 인공지능)', 'Fashion Daraset(패션 데이터세트)', 'AI\u3000Fashion(인공지능 패션)']
2022
['2D Virtual Try-on', 'AI Fashion Convergence', 'Deep Learning Fashion', 'Fashion Digital Transformation', '2D 가상 착의', 'AI 패션 융합', '딥러닝 패션', '패션디지털 전환']
2019
['인공지능', '기술 혁신', '디자인 혁신', '프로세스 혁신', '패션 디자인 프로세스 혁신', 'Artificial Intelligence', 'Technological Innovation', 'Design Innovation', 'Process Innovation', 'Fashion Design Process Innovation']
2021
['4차 산업혁명', '스마트', '인공지능', '패션 제품',

주제어가 아닌 '567', '500', '학술저널'과 같은 데이터들도 같이 출력

- `.infoDetailL >ul >li:nth-of-type(7) > div` 선택자에 주제어 뿐만 아니라 KDC, 자료형태와 같은 데이터들이 같이 있다.
- 태그의 위치가 아닌 텍스트를 인식해서 **주제어**만 가져오는 작업 필요

### 텍스트를 이용해 크롤링 하는 법
- find('태그', [string| class_| id]): 특정 태그 찾는 함수
    - string='문자열' - 해당 문자열을 포함하는 태그 찾음.
    - class_='클래스명' - 해당 클래스명을 포함하는 태그 찾음
    - id='id명' - 해당 id를 포함하는 태그 찾음
- .find_next_sibling() - 현재 태그의 바로 다음 형제 태그(동위 요소)를 가져오는 함수.
    - 다음 형제 태그가 없다면 None 반환합니다.
- .find_previous_sibling() - 현재 태그의 바로 이전 형제 태그(동위 요소)를 가져오는 함수.
    -이전 형제 태그가 없으면 None 반환합니다.
.find_parent() - 현재 태그의 부모 태그를 가져오는 함수.
    - 부모 태그가 없으면 None 반환.

참고👀 find() 함수는 `태그`를 이용해 찾음 `선택자`를 이용해 찾고 싶으면 select(), select_one() 을 이용해야 함


In [18]:
## 발행연도, 주제어 크롤링

header = {
    'User-Agent' : 'Mozilla/5.0 ',
    'Referer' : 'https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
}
for article in articles[:15]:
    title = article.select_one('.title > a').text
    link = 'https://www.riss.kr' + article.select_one('.title > a').get('href')

    # 상세페이지 요청
    response = requests.get(link, headers=header)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # 2. 텍스트를 기준으로 찾기
    press = soup.find('span', string='발행기관').find_next_sibling().text # 바로 옆의 태그를 가져오는 함수
    pyear = soup.find('span', string='발행연도').find_next_sibling().text # 바로 옆의 태그를 가져오는 함수
    subject = soup.find('span', string='주제어').find_next_sibling().text # 바로 옆의 태그를 가져오는 함수
    
    print(press)
    print(pyear)
    sub_data = []
    for sub in subject.split(';'):
        sub_data.append(sub.strip())
    print(sub_data)

한국패션비즈니스학회
2022
['디지털 트랜스포메이션', '패션산업', '인공지능', 'AI 진화 단계', 'digital transformation', 'fashion business', 'artificial intelligence', 'AI evolutionary stage']
한국디자인문화학회
2020
['Artificial Intelligence(인공지능)', 'Fashion Industry(패션산업)', 'Fashion Design (패션 디자인)']
한국전시산업융합연구원
2025
['인공지능', '패션', '인공지능 패션', '텍스트 마이닝', '융합', 'AI', 'Fashion', 'AI Fashion', 'Text Mining', 'Convergence']
한국전시산업융합연구원
2020
['Artificial Intelligence(인공지능)', 'Fashion Styling Service(패션 스타일링 서비스)', 'Stitch Fix(스티치 픽스)', 'Convergence Content(융합 콘텐츠)']
한국디자인문화학회
2023
['AI Learning(인공지능 학습)', 'Fashion Generative AI (패션생성형 인공지능)', 'Fashion Daraset(패션 데이터세트)', 'AI\u3000Fashion(인공지능 패션)']
한국전시산업융합연구원
2022
['2D Virtual Try-on', 'AI Fashion Convergence', 'Deep Learning Fashion', 'Fashion Digital Transformation', '2D 가상 착의', 'AI 패션 융합', '딥러닝 패션', '패션디지털 전환']
커뮤니케이션디자인학회
2019
['인공지능', '기술 혁신', '디자인 혁신', '프로세스 혁신', '패션 디자인 프로세스 혁신', 'Artificial Intelligence', 'Technological Innovation', 'Design Innovation', 'Process Innovation'

AttributeError: 'NoneType' object has no attribute 'find_next_sibling'

**⚠ NoneType에러발생 ⚠**  

→ 항목에서 `주제어`가 없다면 바로 옆의 태그를 찾을 수 없으므로 NoneType 에러 발생.  
→ if 문으로 주제어 항목이 있다면 주제어 반환, 없다면 빈리스트 반환하도록 수정해야 함.

In [19]:
header = {
    'User-Agent' : 'Mozilla/5.0 ',
    'Referer' : 'https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
}
for article in articles[:15]:
    title = article.select_one('.title > a').text
    link = 'https://www.riss.kr' + article.select_one('.title > a').get('href')

    # 상세페이지 요청
    response = requests.get(link, headers=header)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # 2. 텍스트를 기준으로 찾기
    press = soup.find('span', string='발행기관').find_next_sibling().text # 바로 옆의 태그를 가져오는 함수
    pyear = soup.find('span', string='발행연도').find_next_sibling().text # 바로 옆의 태그를 가져오는 함수
    
    ## 수정부분
    if soup.find('span', string='주제어') is None:
        subject = []
    else:
        subject = soup.find('span', string='주제어').find_next_sibling().text.split(';') # 바로 옆의 태그를 가져오는 함수
    
    print(press)
    print(pyear)
    sub_data = []
    for sub in subject:
        sub_data.append(sub.strip())
    print(sub_data)

한국패션비즈니스학회
2022
['디지털 트랜스포메이션', '패션산업', '인공지능', 'AI 진화 단계', 'digital transformation', 'fashion business', 'artificial intelligence', 'AI evolutionary stage']
한국디자인문화학회
2020
['Artificial Intelligence(인공지능)', 'Fashion Industry(패션산업)', 'Fashion Design (패션 디자인)']
한국전시산업융합연구원
2025
['인공지능', '패션', '인공지능 패션', '텍스트 마이닝', '융합', 'AI', 'Fashion', 'AI Fashion', 'Text Mining', 'Convergence']
한국전시산업융합연구원
2020
['Artificial Intelligence(인공지능)', 'Fashion Styling Service(패션 스타일링 서비스)', 'Stitch Fix(스티치 픽스)', 'Convergence Content(융합 콘텐츠)']
한국디자인문화학회
2023
['AI Learning(인공지능 학습)', 'Fashion Generative AI (패션생성형 인공지능)', 'Fashion Daraset(패션 데이터세트)', 'AI\u3000Fashion(인공지능 패션)']
한국전시산업융합연구원
2022
['2D Virtual Try-on', 'AI Fashion Convergence', 'Deep Learning Fashion', 'Fashion Digital Transformation', '2D 가상 착의', 'AI 패션 융합', '딥러닝 패션', '패션디지털 전환']
커뮤니케이션디자인학회
2019
['인공지능', '기술 혁신', '디자인 혁신', '프로세스 혁신', '패션 디자인 프로세스 혁신', 'Artificial Intelligence', 'Technological Innovation', 'Design Innovation', 'Process Innovation'

제대로 된 결과가 나온다!!

### 엑셀파일 저장


In [21]:
import pandas as pd

In [ ]:
header = {
    'User-Agent' : 'Mozilla/5.0 ',
    'Referer' : 'https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
}
# 데이터 저장할 리스트
data = []
for article in articles:
    title = article.select_one('.title > a').text
    link = 'https://www.riss.kr' + article.select_one('.title > a').get('href')

    # 상세페이지 요청
    response = requests.get(link, headers=header)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # 2. 텍스트를 기준으로 찾기
    press = soup.find('span', string='발행기관').find_next_sibling().text # 바로 옆의 태그를 가져오는 함수
    pyear = soup.find('span', string='발행연도').find_next_sibling().text # 바로 옆의 태그를 가져오는 함수
    
    ## 수정부분
    if soup.find('span', string='주제어') is None:
        subject = []
    else:
        subject = soup.find('span', string='주제어').find_next_sibling().text.split(';') # 바로 옆의 태그를 가져오는 함수

    sub_data = []
    for sub in subject:
        sub_data.append(sub.strip())
    
    # 데이터 저장 및 엑셀파일 생성
    # 예외처리는 뺐다..ㅎ
    data.append([title, link, press, pyear, ', '.join(sub_data)])
    df = pd.DataFrame(data, columns=['제목', '링크', '발행기관', '발행연도', '주제어'])
    df.to_excel('RISS_crawlling.xlsx')